In [1]:
import numpy as np,numpy  # 数据处理的库 numpy
import cv2   # 图像处理的库 OpenCv
import sys

In [2]:
# 从text文本中读取特征点
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    # Apply the Affine Transform just found to the src image
    #像素使用线性插值的方式来对RGB颜色通道进行融合，flags=cv2.INTER_LINEAR
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )
    return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))
    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []
    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);
    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]
    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t1Rect, tRect, size)
    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2
    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

#根据特征点中心位置使用最小矩形截取融合后图片并保存
def shape_face(file_path,point_path,save_path):
    img=cv2.imread(file_path)
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    black=np.array([0,0,0])
    mask = cv2.inRange(hsv,black,black)
    #腐蚀膨胀
    erode=cv2.erode(mask,None,iterations=1)
    dilate=cv2.dilate(erode,None,iterations=1)
    #将RGB图转化成二值图
    imgray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,10,255,cv2.THRESH_BINARY)
    contours = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    #boundingRect框图法
    x,y,w,h = cv2.boundingRect(cnt)
#     cv2.rectangle(img, (x, y), (x+w+10, y+h+10), (0, 0, 255), 3)
    img=img[y:y+h,x:x+w]
#   img=cv2.resize(img,size(2400,2400))
    cv2.imwrite(save_path,img)
#     cv2.namedWindow("res",img)
#     cv2.imshow('res',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [6]:
if __name__=='__main__':
    filename1 = '/home/fengchao/桌面/facedata/ave_face.jpg'
    filename2 = '/home/fengchao/桌面/facedata/ave_face.jpg'
    alpha =1 #alpha调节融合程度，数值越小第一张图融合成分越多
    
    # 读取图片
    img1 = cv2.imread(filename1);#模板图
    img2 = cv2.imread(filename2);#融合图
  
    # 将图片像素值转换成浮点数
    img1 = np.float64(img1)
    img2 = np.float64(img2)
    # 读取两张图片的特征点
    points1 = readPoints(filename1 + '.txt')
    points2 = readPoints('/home/fengchao/桌面/facedata/pca/pc1p.txt')

    #创建一个list存放morph后的特征点
    morph_points = []
    #计算alpha权重特征点获取融合特征点
    for i in range(0, len(points1)):
        x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
        y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
        morph_points.append((x,y))
    merge_point_path="/home/fengchao/桌面/facedata/pca/morph_points"+str(alpha)+".txt"
    np.savetxt(merge_point_path,morph_points,fmt='%i %i')
    
    #为模板面孔赋值用于生成基于图片一的图片空间
    morph_img= np.zeros(img1.shape,dtype = img1.dtype)
    src_img= np.zeros(img1.shape,dtype = img1.dtype)
    # 从tri.txt文档中读入面部三角
    with open("/home/fengchao/桌面/facedata/humantri.txt") as file :
        for line in file :
            x,y,z = line.split()
            
            x = int(x)
            y = int(y)
            z = int(z)
            
            t1 = [points1[x], points1[y], points1[z]]
            t2 = [points2[x], points2[y], points2[z]]
            t = [morph_points[x], morph_points[y], morph_points[z]]
            #三角融合算法，对上一步转换后的待融合图片再次取关键点，然后与模特图的关键点一起做三角融合成新的图片
            morphTriangle(img1, img2, morph_img, t1, t2, t, alpha)
    
    # Display Result

    morph_path='/home/fengchao/桌面/facedata/pca/pc1p.jpg'
    cv2.imwrite(morph_path, morph_img)
#     cv2.namedWindow("Morphed Face",0);
#     cv2.imshow("Morphed Face",np.uint8(morph_img))
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     save_path='/home/fengchao/桌面/shapedface'+str(alpha)+'.jpg'
#     #截取融合后的面部并重新设定大小
#     shape_face(morph_path,merge_point_path,save_path)

    

In [5]:
if __name__=='__main__':
    for img_num in range(2,597):
        if img_num==2:
            filename1 = '/home/fengchao/桌面/facedata/78points_shapefree/'+str(img_num-1)+'.jpg'
            path1='/home/fengchao/桌面/facedata/average_face.txt'
            filename2 = '/home/fengchao/桌面/facedata/78points_shapefree/'+str(img_num)+'.jpg'
            path2='/home/fengchao/桌面/facedata/average_face.txt'
        else:
            filename1='/home/fengchao/桌面/facedata/morph_img.jpg'
            path1="/home/fengchao/桌面/facedata/average_face.txt"
            filename2 = '/home/fengchao/桌面/facedata/78points_shapefree/'+str(img_num)+'.jpg'
            path2='/home/fengchao/桌面/facedata/average_face.txt'
        alpha =0.016778523 #alpha调节融合程度，数值越小第一张图融合成分越多,10除以图片数量来计算每张图片所占融合图的比值

        # 读取图片
        img1 = cv2.imread(filename1);#模板图
        img2 = cv2.imread(filename2);#融合图

        # 将图片像素值转换成浮点数
        img1 = np.float64(img1)
        img2 = np.float64(img2)
        # 读取两张图片的特征点
        points1 = readPoints(path1)
        points2 = readPoints(path2)

        #创建一个list存放morph后的特征点            
        morph_points = []
        #计算alpha权重特征点获取融合特征点
        for i in range(0, len(points1)):
            x = ( 1 - alpha ) * points1[i][0] + alpha * points2[i][0]
            y = ( 1 - alpha ) * points1[i][1] + alpha * points2[i][1]
            morph_points.append((x,y))
        merge_point_path="/home/fengchao/桌面/facedata/merge_point.txt"
        np.savetxt(merge_point_path,morph_points,fmt='%i %i')

        #为模板面孔赋值用于生成基于图片一的图片空间
        morph_img= np.zeros(img1.shape,dtype = img1.dtype)
        src_img= np.zeros(img1.shape,dtype = img1.dtype)
        # 从tri.txt文档中读入面部三角
        with open("/home/fengchao/桌面/facedata/humantri.txt") as file :
            for line in file :
                x,y,z = line.split()

                x = int(x)
                y = int(y)
                z = int(z)

                t1 = [points1[x], points1[y], points1[z]]
                t2 = [points2[x], points2[y], points2[z]]
                t = [morph_points[x], morph_points[y], morph_points[z]]
                #三角融合算法，对上一步转换后的待融合图片再次取关键点，然后与模特图的关键点一起做三角融合成新的图片
                morphTriangle(img1, img2, morph_img, t1, t2, t, alpha)

        # Display Result

        morph_path='/home/fengchao/桌面/facedata/morph_img.jpg'
        cv2.imwrite(morph_path, morph_img)
    #     cv2.namedWindow("Morphed Face",0);
    #     cv2.imshow("Morphed Face",np.uint8(morph_img))
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()

    #     save_path='/home/fengchao/桌面/merge results/shapedface/shapedface'+str(alpha)+'.jpg'
    #     #截取融合后的面部并重新设定大小
    #     shape_face(morph_path,merge_point_path,save_path)

    